# Looking at the data from the national incident based reporting system.

National Incident-Based Reporting System, 2016: Extract Files (ICPSR 37066)

https://www.icpsr.umich.edu/web/NACJD/studies/37066#

You need to download the data from the website. It requires you create an account.

In [81]:
import os
import pathlib
import zipfile
import pandas as pd
import re

## Variables of Interest

The field names in the data files do not typically have meaningful names. This maps some of the field names to more meaningful names.


In [45]:
# Field names and short names for variables of interest

variables_info = """
    INCNUM - INCIDENT NUMBER
    INCDATE - INCIDENT DATE
    BH007 - CITY NAME
    BH008 - STATE ABBREVIATION
    V1007 - INCIDENT DATE HOUR
    V1010 - TOTAL OFFENDER SEGMENTS
    V1013 - CLEARED EXCEPTIONALLY
    V20061 - UCR OFFENSE CODE
    V20071 - OFFENSE ATTEMPTED/COMPLETED
    V20081 - OFFENDER(S) SUSPECTED OF USING
    V20111 - LOCATION TYPE
    V20141 - TYPE OF CRIMINAL ACTIVITY
    V20171 - TYPE WEAPON/FORCE INVOLVED
    V20201 - BIAS MOTIVATION
    V4017 - TYPE OF VICTIM
    V4018 - AGE OF VICTIM
    V4019 - SEX OF VICTIM
    V4020 - RACE OF VICTIM
    V4032 - RELATIONSHIP OF VICTIM TO OFFENDER
    V1010 - TOTAL OFFENDER SEGMENTS
    V1009 - TOTAL VICTIM SEGMENTS
"""

fields = {}
for line in variables_info.split("\n"):
    line = line.strip()
    parts = line.split(" - ")
    if len(parts) == 2:
        fields[parts[0]] = parts[1]
        
fields

{'INCNUM': 'INCIDENT NUMBER',
 'INCDATE': 'INCIDENT DATE',
 'BH007': 'CITY NAME',
 'BH008': 'STATE ABBREVIATION',
 'V1007': 'INCIDENT DATE HOUR',
 'V1010': 'TOTAL OFFENDER SEGMENTS',
 'V1013': 'CLEARED EXCEPTIONALLY',
 'V20061': 'UCR OFFENSE CODE',
 'V20071': 'OFFENSE ATTEMPTED/COMPLETED',
 'V20081': 'OFFENDER(S) SUSPECTED OF USING',
 'V20111': 'LOCATION TYPE',
 'V20141': 'TYPE OF CRIMINAL ACTIVITY',
 'V20171': 'TYPE WEAPON/FORCE INVOLVED',
 'V20201': 'BIAS MOTIVATION',
 'V4017': 'TYPE OF VICTIM',
 'V4018': 'AGE OF VICTIM',
 'V4019': 'SEX OF VICTIM',
 'V4020': 'RACE OF VICTIM',
 'V4032': 'RELATIONSHIP OF VICTIM TO OFFENDER',
 'V1009': 'TOTAL VICTIM SEGMENTS'}

## Load Code Values

In [108]:
filename_re = re.compile(r"(.*)\.txt$")
code_line_re = re.compile(r"^(-?[\d\.]+)\s+(.+)")

def read_code_file(code_file):
    codes = {}
    with open(code_file, "r") as fh:
        for line in fh:
            line = line.strip()
            m = code_line_re.match(line)
            if m:
                codes[float(m.group(1))] = m.group(2)
    return codes

def read_codes_dir(codes_dir):
    field_codes = {}
    for f in os.listdir(codes_dir):
        m = filename_re.match(f)
        if m:
            var_name = m.group(1)
            codes = read_code_file(codes_dir + "/" + f)
            field_codes[var_name] = codes
    return field_codes

def get_code_values(codes, values):
    return values.apply(codes.get)

field_codes = read_codes_dir("codes")

## Extract the raw data

This extracts the zip file from the download directory into a data directory.

In [54]:
download_dir = os.path.join(pathlib.Path.home(), "Downloads")
zip_download_file = os.path.join(download_dir, "ICPSR_37066-V2.zip")
national_incident_data_dir = "data/national_incident_data"

# Check that the file is downloaded
if not(os.path.isfile(zip_download_file)):
    raise Exception("Please download the data file!")
    
if not(os.path.isdir(national_incident_data_dir)):
    with zipfile.ZipFile(zip_download_file, "r") as zfh:
        zfh.extractall(national_incident_data_dir)

## Examine Incident File

Examine and explore the incident file.


In [23]:
incident_file = national_incident_data_dir + "/ICPSR_37066/DS0001/37066-0001-Data.tsv"

In [109]:
# Read 1 row to get list of all fields present.
data_1row = pd.read_csv(incident_file, sep="\t", nrows=1)

# Figure out which of the fields in the incident file we want.
fields_to_pull = [c for c in data_1row.columns.values if c in fields]

In [110]:
incident_data = pd.read_csv(incident_file, sep="\t", usecols=fields_to_pull)
incident_data.columns = [fields[c] for c in fields_to_pull]

In [111]:
# Expand code values
for f in fields_to_pull:
    if f in field_codes:
        field_name = fields[f]
        incident_data[field_name] = incident_data[field_name].apply(field_codes[f].get)

In [112]:
# There are > 5 million incidents.
len(incident_data)

5293536

## Examine Number of victims to number of offenders

In [126]:
# 83% are 1 victim to 1 offender.

victim_to_offender_counts = (incident_data
    .groupby(['TOTAL VICTIM SEGMENTS', 'TOTAL OFFENDER SEGMENTS'])
    .size()
    .reset_index()
    .rename(columns={0: 'Count'})
    .sort_values(by='Count', ascending=False)
)
victim_to_offender_counts['Percent'] = 100.0 * victim_to_offender_counts.Count / victim_to_offender_counts.Count.sum()
victim_to_offender_counts['Percent Cumulative'] = victim_to_offender_counts['Percent'].cumsum()

In [128]:
victim_to_offender_counts[victim_to_offender_counts['Percent Cumulative'] <= 99]

,TOTAL VICTIM SEGMENTS,TOTAL OFFENDER SEGMENTS,Count,Percent,Percent Cumulative
1,1,1,4397938,83.081290,83.081290
2,1,2,309035,5.837969,88.919259
40,2,1,297505,5.620156,94.539416
41,2,2,87040,1.644270,96.183685
3,1,3,63430,1.198254,97.381939
71,3,1,44367,0.838135,98.220074
4,1,4,19459,0.367599,98.587674
87,4,1,12103,0.228637,98.816311


## What is the date range for data

In [100]:
incident_data["INCIDENT DATE"].min()

20160101

In [101]:
incident_data["INCIDENT DATE"].max()

20161231

## What Are the Violent Offenses to Look Into?

In [135]:
violent_offense_types = [
    'Simple Assault',
    'Intimidation',
    'Aggravated Assault',
    'Fondling (Indecent Liberties/Child Molesting)',
    'Rape',
    'Kidnaping/Abduction',
    'Statutory Rape',
    'Sexual Assault With An Object',
    'Murder/Nonnegligent Manslaughter',
    'Human Trafficking - Commercial Sex Acts',
    'Human Trafficking - Involuntary Servitude'
]

offense_type_counts = (incident_data
    .groupby("UCR OFFENSE CODE")
    .size()
    .reset_index()
    .rename(columns={0:'Count'})
    .sort_values(by='Count', ascending=False)
)

offense_type_counts['Violent'] = offense_type_counts["UCR OFFENSE CODE"].apply(lambda c: c in violent_offense_types)

In [137]:
# These are the ones to look into for study.
offense_type_counts[offense_type_counts.Violent]

,UCR OFFENSE CODE,Count,Violent
40,Simple Assault,683290,True
24,Intimidation,212097,True
0,Aggravated Assault,187426,True
16,Fondling (Indecent Liberties/Child Molesting),34336,True
36,Rape,33032,True
26,Kidnaping/Abduction,16766,True
28,Murder/Nonnegligent Manslaughter,4416,True
43,Statutory Rape,4379,True
38,Sexual Assault With An Object,3093,True
19,Human Trafficking - Commercial Sex Acts,116,True


In [140]:
# These are the offenses to not consider for study.
# Should some of these be included?
offense_type_counts[~offense_type_counts.Violent]

,UCR OFFENSE CODE,Count,Violent
1,All Other Larceny,630620,False
10,Destruction/Damage/Vandalism of Property,557257,False
12,Drug/Narcotic Violations,500422,False
7,Burglary/Breaking and Entering,448371,False
47,Theft From Motor Vehicle,416884,False
39,Shoplifting,382944,False
27,Motor Vehicle Theft,202107,False
45,Theft From Building,197963,False
15,False Pretenses/Swindle/Confidence Game,121378,False
9,Credit Card/Automatic Teller Machine Fraud,88828,False


## Look further into violent 1 on 1 incidents

In [143]:
v_1v1_incidents = incident_data[
    (incident_data['TOTAL VICTIM SEGMENTS'] == 1) &
    (incident_data['TOTAL OFFENDER SEGMENTS'] == 1) &
    incident_data['UCR OFFENSE CODE'].isin(violent_offense_types)
]

In [146]:
v_1v1_incidents.groupby(['UCR OFFENSE CODE']).size().reset_index().rename(columns={0:'Count'}).sort_values(by='Count', ascending=False)

,UCR OFFENSE CODE,Count
9,Simple Assault,542966
4,Intimidation,174320
0,Aggravated Assault,127917
1,Fondling (Indecent Liberties/Child Molesting),29527
7,Rape,29201
5,Kidnaping/Abduction,11897
10,Statutory Rape,3901
8,Sexual Assault With An Object,2677
6,Murder/Nonnegligent Manslaughter,2672
2,Human Trafficking - Commercial Sex Acts,83


## Examine location to offense type

In [189]:
def related_category_counts(df, x_col, y_col):
    total_count_field = f"{y_col} Total Count"
    total_counts = (df
        .groupby([y_col])
        .size()
        .reset_index()
        .rename(columns={0: total_count_field})
        .sort_values(by=total_count_field, ascending=False)
    )

    stats = (df
        .groupby([x_col, y_col])
        .size()
        .reset_index()
        .rename(columns={0: 'Count'})
        .sort_values(by='Count', ascending=False)
        .pivot(index=y_col, columns=x_col, values='Count')
    )

    stats = pd.merge(
        stats,
        total_counts,
        on=y_col
    )

    stats = stats.sort_values(by=total_count_field, ascending=False)
    return stats

In [190]:
related_category_counts(v_1v1_incidents, 'UCR OFFENSE CODE', 'LOCATION TYPE')

,LOCATION TYPE,Aggravated Assault,Fondling (Indecent Liberties/Child Molesting),Human Trafficking - Commercial Sex Acts,Human Trafficking - Involuntary Servitude,Intimidation,Kidnaping/Abduction,Murder/Nonnegligent Manslaughter,Rape,Sexual Assault With An Object,Simple Assault,Statutory Rape,LOCATION TYPE Total Count
35,Residence/Home,75402.0,19564.0,19.0,2.0,109048.0,7907.0,1472.0,20702.0,1867.0,355252.0,2716.0,593951
24,Highway/Road/Alley/Street/Sidewalk,24224.0,1310.0,4.0,NaN,14611.0,1667.0,628.0,1380.0,104.0,53737.0,106.0,97771
31,Other/unknown,4239.0,1651.0,2.0,4.0,11409.0,428.0,103.0,1780.0,168.0,18116.0,346.0,38246
33,Parking Lot/Garage,6209.0,485.0,NaN,NaN,4696.0,452.0,137.0,711.0,39.0,20989.0,82.0,33800
39,School--elementary/secondary,1413.0,1547.0,1.0,NaN,5505.0,139.0,NaN,340.0,68.0,15897.0,105.0,25015
25,Hotel/Motel/Etc.,1830.0,467.0,51.0,1.0,1409.0,404.0,29.0,1216.0,65.0,8811.0,99.0,14382
7,Bar/Nightclub,2344.0,223.0,NaN,1.0,1045.0,45.0,28.0,190.0,24.0,8447.0,4.0,12351
37,Restaurant,1093.0,324.0,NaN,NaN,3159.0,76.0,16.0,88.0,20.0,6438.0,9.0,11223
18,Drug Store/Drs Office/Hospital,930.0,483.0,1.0,NaN,2207.0,43.0,15.0,303.0,38.0,6745.0,19.0,10784
27,Jail/Prison/Penitentiary/Corrections Facility,1568.0,261.0,NaN,NaN,938.0,9.0,13.0,66.0,39.0,6984.0,9.0,9887


## Load Victim Data

In [168]:
victim_file = national_incident_data_dir + "/ICPSR_37066/DS0002/37066-0002-Data.tsv"

In [171]:
# Read 1 row to get list of all fields present.
v_data_1row = pd.read_csv(victim_file, sep="\t", nrows=1)

# Figure out which of the fields in the victim file we want.
v_fields_to_pull = [c for c in v_data_1row.columns.values if c in fields]

In [172]:
victim_data = pd.read_csv(victim_file, sep="\t", usecols=v_fields_to_pull)
victim_data.columns = [fields[c] for c in v_fields_to_pull]

In [173]:
# Expand code values
for f in v_fields_to_pull:
    if f in field_codes:
        field_name = fields[f]
        victim_data[field_name] = victim_data[field_name].apply(field_codes[f].get)

In [181]:
victim_data.sample(3)

,INCIDENT NUMBER,INCIDENT DATE,CITY NAME,STATE ABBREVIATION,INCIDENT DATE HOUR,TOTAL VICTIM SEGMENTS,TOTAL OFFENDER SEGMENTS,CLEARED EXCEPTIONALLY,TYPE OF VICTIM,AGE OF VICTIM,SEX OF VICTIM,RACE OF VICTIM,RELATIONSHIP OF VICTIM TO OFFENDER,UCR OFFENSE CODE,OFFENSE ATTEMPTED/COMPLETED,OFFENDER(S) SUSPECTED OF USING,LOCATION TYPE,TYPE OF CRIMINAL ACTIVITY,TYPE WEAPON/FORCE INVOLVED,BIAS MOTIVATION
480903,9CA4KL68D39G,20160324,ASPEN,CO,21,7,1,Not applicable,Business,Not applicable,Not applicable,Not applicable,Not applicable,Destruction/Damage/Vandalism of Property,Completed,Not applicable,Highway/Road/Alley/Street/Sidewalk,Not applicable,Undetermined,None
2324029,2-MR3P1TX I,20160622,BILOXI,MS,20,1,1,Not applicable,Individual,None,Female,Black or African American,Unknown/Missing/DNR,Simple Assault,Completed,Not applicable,Highway/Road/Alley/Street/Sidewalk,None/Unknown Gang Involvement,"Personal Weapons (hands, feet, teeth, etc.)",None
4009889,2W1N0UKU6XXA,20160124,NASHVILLE METROPOLITAN,TN,0,1,1,Not applicable,Business,Not applicable,Not applicable,Not applicable,Not applicable,Credit Card/Automatic Teller Machine Fraud,Completed,Not applicable,Residence/Home,Not applicable,Undetermined,None


In [178]:
victim_data_v_1v1 = victim_data[
    (victim_data['TOTAL VICTIM SEGMENTS'] == 1) &
    (victim_data['TOTAL OFFENDER SEGMENTS'] == 1) &
    victim_data['UCR OFFENSE CODE'].isin(violent_offense_types)
]

In [192]:
related_category_counts(victim_data_v_1v1, 'UCR OFFENSE CODE', 'RELATIONSHIP OF VICTIM TO OFFENDER')

,RELATIONSHIP OF VICTIM TO OFFENDER,Aggravated Assault,Fondling (Indecent Liberties/Child Molesting),Human Trafficking - Commercial Sex Acts,Human Trafficking - Involuntary Servitude,Intimidation,Kidnaping/Abduction,Murder/Nonnegligent Manslaughter,Rape,Sexual Assault With An Object,Simple Assault,Statutory Rape,RELATIONSHIP OF VICTIM TO OFFENDER Total Count
6,Victim was Boyfriend/Girlfriend,26252.0,758.0,8.0,NaN,23057.0,4531.0,204.0,3107.0,167.0,141228.0,988.0,200300
3,Unknown/Missing/DNR,25777.0,5155.0,10.0,4.0,33849.0,1598.0,1238.0,6278.0,562.0,60912.0,559.0,135942
4,Victim was Acquaintance,17359.0,6375.0,31.0,2.0,31913.0,955.0,349.0,7596.0,584.0,69335.0,1052.0,135551
19,Victim was Otherwise Known,11353.0,3341.0,12.0,NaN,24988.0,710.0,164.0,2569.0,255.0,49781.0,404.0,93577
22,Victim was Spouse,9254.0,129.0,NaN,NaN,9152.0,1210.0,172.0,735.0,43.0,57249.0,2.0,77946
26,Victim was Stranger,11315.0,1868.0,11.0,NaN,13753.0,728.0,146.0,1935.0,132.0,31263.0,70.0,61221
20,Victim was Parent,3772.0,27.0,NaN,NaN,5308.0,176.0,54.0,18.0,4.0,29749.0,NaN,39108
7,Victim was Child,3902.0,2254.0,NaN,NaN,2526.0,568.0,74.0,939.0,181.0,18294.0,83.0,28821
13,Victim was Friend,3910.0,1580.0,1.0,NaN,4060.0,200.0,78.0,2310.0,193.0,13502.0,305.0,26139
21,Victim was Sibling,3457.0,1233.0,NaN,NaN,3056.0,48.0,31.0,522.0,67.0,17574.0,56.0,26044


In [193]:
related_category_counts(victim_data_v_1v1, 'UCR OFFENSE CODE', 'SEX OF VICTIM')

,SEX OF VICTIM,Aggravated Assault,Fondling (Indecent Liberties/Child Molesting),Human Trafficking - Commercial Sex Acts,Human Trafficking - Involuntary Servitude,Intimidation,Kidnaping/Abduction,Murder/Nonnegligent Manslaughter,Rape,Sexual Assault With An Object,Simple Assault,Statutory Rape,SEX OF VICTIM Total Count
0,Female,60298.0,24503.0,79.0,8.0,110081.0,9973.0,713.0,28589.0,2344.0,341560.0,3579.0,581727
1,Male,67025.0,4856.0,2.0,NaN,62460.0,1889.0,1944.0,612.0,323.0,198379.0,322.0,337812
2,Unknown/missing/DNR,594.0,168.0,2.0,NaN,1779.0,35.0,15.0,NaN,10.0,3027.0,NaN,5630
